In [7]:
!pip install ipywidgets pytz

In [8]:
import json
from datetime import datetime, timedelta, date, time
import pytz
import ipywidgets as widgets
from IPython.display import display, clear_output

In [9]:
# ---------------------- Event class ----------------------
class Event:
    def __init__(self, title, start, end):
        self.title = title
        self.start = start
        self.end = end

    def to_dict(self):
        return {
            "title": self.title,
            "start": self.start.isoformat(),
            "end": self.end.isoformat()
        }

In [10]:
# ---------------------- Calendar App class ----------------------
class CalendarApp:
    def __init__(self):
        self.events = []
        self.load()

    def load(self):
        try:
            with open("calendar.json", "r") as f:
                for e in json.load(f):
                    self.events.append(Event(e["title"], datetime.fromisoformat(e["start"]), datetime.fromisoformat(e["end"])))
        except:
            pass

    def save(self):
        with open("calendar.json", "w") as f:
            json.dump([e.to_dict() for e in self.events], f)

    def add(self, title, start, end):
        for e in self.events:
            if e.start < end and start < e.end:
                return False
        self.events.append(Event(title, start, end))
        self.save()
        return True

    def delete(self, title, day):
        before = len(self.events)
        self.events = [e for e in self.events if not (e.title == title and e.start.date() == day)]
        self.save()
        return len(self.events) < before

    def on_day(self, day):
        return [e for e in self.events if e.start.date() == day]

    def free_slot(self, mins, day, tz):
        t = tz.localize(datetime.combine(day, time(8, 0)))
        end = tz.localize(datetime.combine(day, time(23, 59)))
        while t + timedelta(minutes=mins) <= end:
            if all(t + timedelta(minutes=mins) <= e.start or t >= e.end for e in self.events):
                return t, t + timedelta(minutes=mins)
            t += timedelta(minutes=15)
        return None, None

    def all_free_slots(self, mins, day, tz):
        slots = []
        t = tz.localize(datetime.combine(day, time(8, 0)))
        end_of_day = tz.localize(datetime.combine(day, time(23, 59)))
        while t + timedelta(minutes=mins) <= end_of_day:
            candidate_end = t + timedelta(minutes=mins)
            overlap = any(e.start < candidate_end and t < e.end for e in self.events)
            if not overlap:
                slots.append((t, candidate_end))
                t = candidate_end
            else:
                t += timedelta(minutes=15)
        return slots


In [11]:
# ---------------------- Widgets ----------------------
calendar = CalendarApp()

# Styled title without emoji to avoid UnicodeEncodeError
display(widgets.HTML(
    "<h2 style='text-align:center; color:#2c3e50; font-family:sans-serif; padding:10px; "
    "background:#e3f2fd; border-radius:10px;'><b>Calendar Application</b></h2>"
))

title = widgets.Text(description="Title:")
date_picker = widgets.DatePicker(description="Date:")
sh = widgets.Dropdown(options=list(range(1, 13)), value=9, description="Start Hr")
sm = widgets.Dropdown(options=list(range(0, 60, 5)), value=0, description="Min")
sap = widgets.Dropdown(options=["AM", "PM"], value="AM", description="AM/PM")
eh = widgets.Dropdown(options=list(range(1, 13)), value=10, description="End Hr")
em = widgets.Dropdown(options=list(range(0, 60, 5)), value=0, description="Min")
eap = widgets.Dropdown(options=["AM", "PM"], value="AM", description="AM/PM")

tz = widgets.Dropdown(
    options=[
        
        ("Eastern Daylight Time (EDT)", "US/Eastern"),
        ("Central Daylight Time (CDT)", "US/Central"),
        ("Mountain Daylight Time (MDT)", "US/Mountain"),
        ("Pacific Daylight Time (PDT)", "US/Pacific"),
        ("India Standard Time (IST)", "Asia/Kolkata"),
        ("Greenwich Mean Time (GMT)", "Etc/GMT")
    ],
    description="Timezone:"
)

slot_duration = widgets.Dropdown(
    options=[("30 minutes", 30), ("60 minutes", 60), ("90 minutes", 90), ("120 minutes", 120), ("150 minutes", 150),("180 minutes", 180)],
    description="Slot Size:"
)

delete_title = widgets.Text(description="Del Title:")
delete_date = widgets.DatePicker(description="Del Date:")

btn_layout = widgets.Layout(width='160px', height='40px', border_radius='10px')
add_btn = widgets.Button(description="Add Event", style={'button_color': '#28a745'}, layout=btn_layout)
view_btn = widgets.Button(description="View Events", style={'button_color': '#17a2b8'}, layout=btn_layout)
slot_btn = widgets.Button(description="Next Slot", style={'button_color': '#ffc107'}, layout=btn_layout)
all_slots_btn = widgets.Button(description="All Free Slots", style={'button_color': '#007bff'}, layout=btn_layout)
remaining_btn = widgets.Button(description="Remaining Today", style={'button_color': '#6f42c1'}, layout=btn_layout)
delete_btn = widgets.Button(description="Delete Event", style={'button_color': '#dc3545'}, layout=btn_layout)

out = widgets.Output()

def to24(h, ap):
    return h if ap == "AM" and h != 12 else (h + 12 if ap == "PM" and h != 12 else 0)

def clear_inputs():
    title.value = ""
    date_picker.value = None
    sh.value, sm.value, sap.value = 9, 0, "AM"
    eh.value, em.value, eap.value = 10, 0, "AM"

def on_add(_):
    with out:
        clear_output()
        if not date_picker.value:
            print("Please select a date.")
            return
        zone = pytz.timezone(tz.value)
        s = zone.localize(datetime.combine(date_picker.value, time(to24(sh.value, sap.value), sm.value)))
        e = zone.localize(datetime.combine(date_picker.value, time(to24(eh.value, eap.value), em.value)))
        if s >= e:
            print("End time must be after start time.")
            return
        if calendar.add(title.value, s, e):
            print("Event added successfully.")
            clear_inputs()
        else:
            print("This time overlaps with another event.")

def on_view(_):
    with out:
        clear_output()
        if not date_picker.value:
            print("Please select a date.")
            return
        events = sorted(calendar.on_day(date_picker.value), key=lambda e: e.start)
        if not events:
            print("No events on this day.")
        else:
            print(f"Events on {date_picker.value}:")
            for e in events:
                print(f"{e.start.strftime('%I:%M %p')} - {e.end.strftime('%I:%M %p')} | {e.title}")

def on_slot(_):
    with out:
        clear_output()
        if not date_picker.value:
            print("Please select a date.")
            return
        zone = pytz.timezone(tz.value)
        start, end = calendar.free_slot(slot_duration.value, date_picker.value, zone)
        if start:
            print(f"Next available slot: {start.strftime('%I:%M %p')} - {end.strftime('%I:%M %p')}")
        else:
            print("No free slot available for this duration.")

def on_all_slots(_):
    with out:
        clear_output()
        if not date_picker.value:
            print("Please select a date.")
            return
        zone = pytz.timezone(tz.value)
        slots = calendar.all_free_slots(slot_duration.value, date_picker.value, zone)
        if not slots:
            print("No available time slots for this duration.")
        else:
            print(f"All available {slot_duration.value}-minute slots on {date_picker.value}:")
            for s, e in slots:
                print(f"{s.strftime('%I:%M %p')} - {e.strftime('%I:%M %p')}")

def on_remaining(_):
    with out:
        clear_output()
        zone = pytz.timezone(tz.value)
        now = datetime.now(pytz.utc).astimezone(zone)
        today = now.date()
        remaining = sorted([e for e in calendar.on_day(today) if e.start >= now], key=lambda e: e.start)
        if not remaining:
            print("No remaining events for today.")
        else:
            print(f"Remaining events for today ({today}):")
            for e in remaining:
                print(f"{e.start.strftime('%I:%M %p')} - {e.end.strftime('%I:%M %p')} | {e.title}")

def on_delete(_):
    with out:
        clear_output()
        if not delete_title.value or not delete_date.value:
            print("Provide title and date to delete.")
            return
        if calendar.delete(delete_title.value, delete_date.value):
            print("Event deleted.")
        else:
            print("No matching event found.")
            
#  buttons           
add_btn.on_click(on_add)
view_btn.on_click(on_view)
slot_btn.on_click(on_slot)
all_slots_btn.on_click(on_all_slots)
remaining_btn.on_click(on_remaining)
delete_btn.on_click(on_delete)


# UI
display(title, date_picker)
display(widgets.HBox([sh, sm, sap]))
display(widgets.HBox([eh, em, eap]))
display(tz, slot_duration)
display(widgets.VBox([
    widgets.HBox([add_btn, view_btn, slot_btn]),
    widgets.HBox([all_slots_btn, remaining_btn, delete_btn])
]))

display(widgets.HTML("<hr><h4 style='color:#c0392b; font-weight:bold;'>Delete Event</h4>"))
display(widgets.HBox([delete_title, delete_date]))

display(out)


HTML(value="<h2 style='text-align:center; color:#2c3e50; font-family:sans-serif; padding:10px; background:#e3f…

Text(value='', description='Title:')

DatePicker(value=None, description='Date:', step=1)

Dropdown(description='Timezone:', options=(('Eastern Daylight Time (EDT)', 'US/Eastern'), ('Central Daylight T…

Dropdown(description='Slot Size:', options=(('30 minutes', 30), ('60 minutes', 60), ('90 minutes', 90), ('120 …

HTML(value="<hr><h4 style='color:#c0392b; font-weight:bold;'>Delete Event</h4>")

Output()